In [1]:
import os
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

import time
import glob
import tables as tb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import invisible_cities.icaro.mpl_functions as mpl
from matplotlib.pyplot import figure, show
from functools import reduce
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from itertools import product, combinations
%matplotlib inline
%load_ext autoreload
%autoreload 2

plt.rcParams["figure.figsize"] = 10, 8
plt.rcParams["font.size"     ] = 14


2017-08-21 14:58:41


In [10]:
from invisible_cities.evm.event_model import Voxel 
from invisible_cities.reco.paolina_functions import make_track_graphs 
from invisible_cities.reco.paolina_functions import shortest_paths
from invisible_cities.reco.paolina_functions import find_extrema

In [ ]:
def make_track_graphs(voxels           : Voxel,
                      voxel_dimensions : np.ndarray,
                      contiguity       : float = 1) ->Sequence[Graph]:
    """Creates a graph where the voxels are the nodes and the edges are any
    pair of neighbour voxel. Two voxels are considered to be neighbours if
    their distance normalized to their size is smaller than a
    contiguity factor .

    """

    def neighbours(va : Voxel, vb : Voxel, scale : float = 1.0) ->bool:
        return ((abs(va.pos - vb.pos) / voxel_dimensions) < contiguity).all()

    voxel_graph = nx.Graph()
    voxel_graph.add_nodes_from(voxels)
    for va, vb in combinations(voxels, 2):
        if neighbours(va, vb):
            voxel_graph.add_edge(va, vb, distance = np.linalg.norm(va.pos - vb.pos))

    return tuple(nx.connected_component_subgraphs(voxel_graph))


In [ ]:
def make_track_graphs(voxels, voxel_dimensions):

    def neighbours(va, vb):
        return ((abs(va.pos - vb.pos) / voxel_dimensions) < 1.5).all()

    voxel_graph = nx.Graph()
    voxel_graph.add_nodes_from(voxels)
    for va, vb in combinations(voxels, 2):
        if neighbours(va, vb):
            voxel_graph.add_edge(va, vb,
                                 distance = np.linalg.norm(va.pos - vb.pos))

    return tuple(nx.connected_component_subgraphs(voxel_graph))


In [15]:
voxel_spec = ((10,10,10,  1000),
                  (10,10,11,     1),
                  (10,10,12,     2),
                  (10,10,13,     4),
                  (10,10,14,     8),
                  (10,10,15,    16),
                  (10,11,15,    32),
                  (10,12,15,    64),
                  (10,13,15,   128),
                  (10,14,15,   265),
                  (10,15,15,   512),
                  (11,15,15,   256),
                  (12,15,15,   128),
                  (13,15,15,    64),
                  (14,15,15,    32),
                  (15,15,15,    16),
                  (16,16,16,     8),
                  (17,17,17,     4),
                  (18,18,18,     2),
                  (19,19,19,     1),
                  (20,20,20,  2000),
    )
voxels = [Voxel(x,y,z, E) for (x,y,z,E) in voxel_spec]
    

In [16]:
voxels

[<[10, 10, 10] 1000>,
 <[10, 10, 11] 1>,
 <[10, 10, 12] 2>,
 <[10, 10, 13] 4>,
 <[10, 10, 14] 8>,
 <[10, 10, 15] 16>,
 <[10, 11, 15] 32>,
 <[10, 12, 15] 64>,
 <[10, 13, 15] 128>,
 <[10, 14, 15] 265>,
 <[10, 15, 15] 512>,
 <[11, 15, 15] 256>,
 <[12, 15, 15] 128>,
 <[13, 15, 15] 64>,
 <[14, 15, 15] 32>,
 <[15, 15, 15] 16>,
 <[16, 16, 16] 8>,
 <[17, 17, 17] 4>,
 <[18, 18, 18] 2>,
 <[19, 19, 19] 1>,
 <[20, 20, 20] 2000>]

In [19]:
tracks = make_track_graphs(voxels, np.array([1,1,1]), contiguity=1.6)

In [20]:
len(tracks)

1

In [7]:
distances = shortest_paths(tracks[0])

In [11]:
distances
extrema = find_extrema(distances)

In [12]:
extrema

(<[10, 10, 10] 1000>, <[20, 20, 20] 2000>)

In [13]:
voxels[ 0]

<[10, 10, 10] 1000>

In [14]:
voxels[ -1]

<[20, 20, 20] 2000>

In [ ]:
track, = make_track_graphs(voxels, np.array([1,1,1]))
    distances = shortest_paths(track)
    extrema = find_extrema(distances)